In [ ]:
import sys
!{sys.executable} -m pip install numpy joblib pandas pandas-datareader scikit-learn pandas-ta statsmodels plotly matplotlib statsmodels
!{sys.executable} -m pip install yfinance==0.2.31

In [ ]:
from joblib import Memory
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

cachedir = "./yfinance_cache" 
memory = Memory(cachedir, verbose=0)


@memory.cache
def cached_yf_download(*args, **kwargs):
    return yf.download(*args, **kwargs)


@memory.cache
def cached_tickers(dt):
    sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
    sp500["Symbol"] = sp500["Symbol"].str.replace(".", "-")
    symbols_list = sp500["Symbol"].unique().tolist()

    QQQ = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100#Components")[4]
    QQQ["Ticker"] = QQQ["Ticker"].str.replace(".", "-")
    nasdaq_symbols_list = QQQ["Ticker"].unique().tolist()

    symbols_list.extend(nasdaq_symbols_list)
    symbols_list.extend(["SPY","IVV","VOO","VTI","QQQ","VEA","IEFA","VTV","BND","VUG","AGG","IWF","IJR","IJH","IEMG","VWO","VIG","IWM","VXUS","VO","VGT","XLK","GLD","IWD","BNDX","XXXX","GBTC","SMH","IBIT","VGT","XLK","BITO"])
    symbols_list = list(set(symbols_list))

    return symbols_list


# end_date = '2023-11-21'
end_date = pd.Timestamp.now().floor("60T")
start_date = (pd.to_datetime(end_date) - pd.DateOffset(1600)).floor("60T")
print(start_date, end_date)

symbols_list = cached_tickers("2024-04-12")

og_df = cached_yf_download(
    tickers=symbols_list, start=start_date, end=end_date, interval="1d", auto_adjust=True
).stack()

og_df.index.names = ["date", "ticker"]

og_df.columns = og_df.columns.str.lower()

og_df['dollar_volume'] = (og_df['close']*og_df['volume'])/1e6

all_tickers_df = og_df

In [ ]:
data = all_tickers_df

data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(1*30, min_periods=12).mean().stack())
data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))
data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

In [ ]:
import math
import datetime
profits = None
buys = {}

for symbol in data.head(150).index.unique(1):
    rsi_buys=[]
    rsi_sells=[]
    
    df = all_tickers_df.xs(symbol, level=1).drop_duplicates()

    df['200sma'] = df['close'].rolling(window=200).mean()
    
    df['price_change']=df['close'].pct_change()
    df['Upmove'] = df['price_change'].apply(lambda x: x if x > 0 else 0)
    df['Downmove'] = df['price_change'].apply(lambda x: abs(x) if x < 0 else 0)
    df['avg_up'] = df['Upmove'].ewm(span=19).mean()
    df['avg_down'] = df['Downmove'].ewm(span=19).mean()
    df = df.dropna()
    df['RS'] = df['avg_up']/df['avg_down']
    df['RSI'] = df['RS'].apply(lambda x: 100-(100/(x+1)))
    
    def rsi_buy(df):
        return (df['close'] > df['200sma']) and df['RSI'] < 30
            
    
    df['RSI_Buy'] = df.apply(rsi_buy, axis=1)
    df['Periods_Since_Buy'] = df.loc[df['RSI_Buy']].index.to_series().diff().fillna(0)
    
    def rsi_sell(window):
        open_position = 0
        current_value = window.iloc[len(window)-1]
        blew_rsi = False
        
        for i in range(len(window)):
            old_value = window.iloc[len(window)-i-1]
            if old_value['RSI_Buy'] == False:
                continue
            open_position = i
            if current_value['RSI'] > 40:
                blew_rsi=True
    
        if open_position >= 10 or blew_rsi:
            return True
    
        return False
    
    results=[]
    for i in range(len(df)):
        if (i < 10):
            results.append(False)
            continue
        window_df = df[max(i - 10+1,0):i+1]
        result = rsi_sell(window_df)
        results.append(result)
    
    df['RSI_Sell'] = results
    df['tx_price'] = df['open'].shift(-1)
    
    in_buy_phase = False
    
    def filter_signals(row):
        global in_buy_phase
        if row['RSI_Buy'] == True and not in_buy_phase:
            in_buy_phase = True
            return True
        if row['RSI_Sell'] == True and in_buy_phase:
            in_buy_phase = False
            return True
        return False
    
    df['keep'] = df.apply(filter_signals, axis=1)
    # print(df[df['keep']].count(numeric_only=True))
    filtered_df = df[df['keep']].drop(columns='keep')
    
    filtered_df.drop(['high', 'low','price_change','volume','dollar_volume','avg_up','avg_down','Upmove','Downmove','dollar_vol_rank','200sma','RS'], inplace=True, axis=1)
    
    filtered_df[filtered_df['RSI_Buy'] | filtered_df['RSI_Sell']]
    
    latest_buy = filtered_df[filtered_df['RSI_Buy']].tail(1)
    if (len(latest_buy) == 1):
        d = latest_buy.iloc[0].name.date()
        # print(f"{symbol}: {d}")
        now = datetime.datetime.now().date()
        isWithinDays = (now-d).days <= 10
        if (isWithinDays):
            print(f"{symbol}: {(d - now).days}: {d}")
            buys[symbol] = filtered_df
            
    if profits is None:
        profits = {}
    cash = 10000
    shares = None
    profits[symbol] = 0
    buy_price = None
    for index, row in filtered_df.iterrows():
        if row['RSI_Buy'] == True:
            buy_price = row['tx_price']
            if (cash > buy_price):
                shares = math.floor(cash/buy_price)
                cost = shares * buy_price
                cash -= cost
        elif row['RSI_Sell'] == True and shares is not None:
            cash +=  shares * row['tx_price']
            profits[symbol] = cash-10000
            shares = None
with pd.option_context('display.float_format', '{:0.2f}'.format):
    for buy in buys: 
        print(buy)
        print(buys[buy])

In [ ]:
if profits is not None:
    profits_df = pd.DataFrame.from_dict(profits, orient='index')
    profits_df.columns=['profit']
    profits_df
    print(profits_df['profit'].sum())

In [ ]:
profits = None
buys = {}

def plot_signals(df, stock_name):
    plt.figure(figsize=(12,5))
    plt.title(f'{stock_name} Buy and Sell Signals')
    plt.plot(df['close'], label='Close Price', color='blue', alpha=0.35)
    plt.plot(df['SMA_200'], label='200 Day SMA', color='red', alpha=0.35)
    plt.scatter(df[df['Buy_Signal']].index, df[df['Buy_Signal']]['close'], color='green', label='Buy Signal', marker='^', alpha=1)
    plt.scatter(df[df['Sell_Signal']].index, df[df['Sell_Signal']]['close'], color='red', label='Sell Signal', marker='v', alpha=1)
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.legend(loc='upper left')
    plt.show()

def calculate_indicators(df):
    # Calculate 200 SMA
    df['SMA_200'] = df['close'].rolling(window=200).mean()

    # Calculate RSI
    df['price_change'] = df['close'].pct_change()
    df['Upmove'] = df['price_change'].apply(lambda x: x if x > 0 else 0)
    df['Downmove'] = df['price_change'].apply(lambda x: abs(x) if x < 0 else 0)
    df['avg_up'] = df['Upmove'].ewm(span=19).mean()
    df['avg_down'] = df['Downmove'].ewm(span=19).mean()
    df = df.dropna()
    df['RS'] = df['avg_up'] / df['avg_down']
    df['RSI'] = df['RS'].apply(lambda x: 100 - (100 / (x + 1)))

    return df

def generate_signals(df):
    # Initialize columns for 'Buy_Signal' and 'Sell_Signal'
    df['Buy_Signal'] = False
    df['Sell_Signal'] = False
    buy_date = None

    for i in range(2, df.shape[0]):
        # Buy signal: RSI drops below 30 and the daily close is above the 200 SMA
        if df['RSI'].iloc[i] < 30 and df['close'].iloc[i] > df['SMA_200'].iloc[i]:
            if buy_date is None:  # Only one 'buy signal' shall be active at any given time
                df['Buy_Signal'].iloc[i] = True
                buy_date = df.index[i]

        # Sell signal: A trade has been open for 10 or more days and the RSI has exceeded 40
        if buy_date is not None and (df.index[i] - buy_date).days >= 10 and df['RSI'].iloc[i] > 40:
            df['Sell_Signal'].iloc[i] = True
            buy_date = None  # Reset buy_date

    return df

# Assuming df is the DataFrame containing the top 150 most liquid stocks


for symbol in data.index.unique(1):
    rsi_buys=[]
    rsi_sells=[]
    
    df = all_tickers_df.xs(symbol, level=1).drop_duplicates()
    df = calculate_indicators(df)
    df = generate_signals(df)
    
    filtered_df = [df['Buy_Signal'] | df['Sell_Signal']]
    
    latest_buy = df[df['Buy_Signal']].tail(1)
    if (len(latest_buy) == 1):
        d = latest_buy.iloc[0].name.date()
        # print(f"{symbol}: {d}")
        now = datetime.datetime.now().date()
        isWithinDays = (now-d).days <= 10
        if (isWithinDays):
            print(f"{symbol}: {(d - now).days}: {d}")
            buys[symbol] = filtered_df
            plot_signals(df, symbol)

# print(buys)